In [ ]:
!pip install pyreadr pandas openpyxl
!pip install --upgrade lightgbm
!pip install lifelines

# ***Data Preparation***

**step 1**

In [ ]:
import pyreadr
import pandas as pd

in_path = "/content/IMvigor210CoreBiologies.Rdata"
out_prefix = "/content/IMvigor210_data"

result = pyreadr.read_r(in_path)

with pd.ExcelWriter(f"{out_prefix}.xlsx") as xw:
    for name, df in result.items():
        csv_path = f"{out_prefix}_{name}.csv"
        df.to_csv(csv_path, index=False)

        sheet = name[:31]
        df.to_excel(xw, sheet_name=sheet, index=False)

In [ ]:
#sampleID repair
import pandas as pd

anno = pd.read_csv("/content/IMvigor210_data_annoData.csv")
expr = pd.read_csv("/content/IMvigor210_data_expreSet.csv")
pheno = pd.read_csv("/content/IMvigor210_data_phenoData.csv")

sample_ids = list(expr.columns)

pheno.insert(0, "SampleID", sample_ids)

expr_T = expr.T.copy()
expr_T.index.name = "SampleID"
expr_T.reset_index(inplace=True)

pheno.to_csv("IMvigor210_pheno_fixed.csv", index=False)
expr_T.to_csv("IMvigor210_expr_fixed.csv", index=False)

In [ ]:
import pandas as pd

print("=== Read pheno_fixed ===")
pheno_fixed = pd.read_csv("IMvigor210_pheno_fixed.csv")
print(pheno_fixed.head())
print("Shape:", pheno_fixed.shape)
print("\nColumns:", pheno_fixed.columns.tolist())

print("\n=== Read expr_fixed ===")
expr_fixed = pd.read_csv("IMvigor210_expr_fixed.csv")
print(expr_fixed.head())
print("Shape:", expr_fixed.shape)
print("\nColumns (first 20):", expr_fixed.columns.tolist()[:20], "...")


print("\n=== Expression Matrix Structure Explanation ===")
print(f"Rows (samples): {expr_fixed.shape[0]}")
print(f"Columns (1 SampleID + {expr_fixed.shape[1]-1} gene-expression columns)")
print("Note: Column names '0,1,2,3,...' represent gene indices (gene rownames were lost when exporting Rdata).")


preview_cols = ["SampleID"] + expr_fixed.columns.tolist()[1:6]
preview = expr_fixed[preview_cols].head()

print("\n=== Expression Matrix Preview (SampleID + first 5 genes) ===")
print(preview.to_string(index=False))


In [ ]:
import pyreadr

result = pyreadr.read_r("/content/IMvigor210CoreBiologies.Rdata")

for k,v in result.items():
    print("====", k, v.shape)
    print(v.head())


In [ ]:
gene_anno = result["annoData"]   # Revise to right name
gene_anno.index = gene_anno.index.astype(str)

# expr rows (gene indices)
expr.index = expr.index.astype(str)

# 合并 gene annotation
expr_annotated = expr.merge(gene_anno, left_index=True, right_index=True)

In [ ]:
result.keys()


In [ ]:
for k, df in result.items():
    print("====", k, df.shape)
    print(df.head())


In [ ]:
import pandas as pd
import pyreadr

result = pyreadr.read_r("/content/IMvigor210CoreBiologies.Rdata")

expr_raw = result["expreSet"]    # 31286 x 348
anno     = result["annoData"]    # 31286 x 6

print(expr_raw.shape, anno.shape)

#  Ensure row indices are aligned
expr_raw = expr_raw.sort_index()
anno     = anno.sort_index()

print("Index equal?", expr_raw.index.equals(anno.index))

# Use the gene symbols from annoData as new row names
# Both columns: "symbol" / "Symbol" contain gene names, either can be used; here we use the capitalized 'Symbol'
gene_symbols = anno["Symbol"].astype(str)

expr_raw.index = gene_symbols

print("=== expr_raw with gene symbols (first 5 rows) ===")
print(expr_raw.head())


In [ ]:
# Transpose: rows = SampleID, columns = gene symbol
expr_T = expr_raw.T.copy()
expr_T.index.name = "SampleID"
expr_T.reset_index(inplace=True)

expr_T.to_csv("IMvigor210_expr_fixed_with_symbols.csv", index=False)

print("Shape of expr_T:", expr_T.shape)
print("\nColumns (first 10) of expr_T:")
print(expr_T.columns[:10].tolist())


In [ ]:
import pandas as pd
import pyreadr

# 1. Reload the Rdata file
result = pyreadr.read_r("/content/IMvigor210CoreBiologies.Rdata")

# Original expression matrix (rows = feature index)
expr_raw = result["expreSet"]

# Gene annotation (rows = feature index)
anno = result["annoData"]

print("Expr shape:", expr_raw.shape)   # (31286, 348)
print("Anno shape:", anno.shape)       # (31286, 6)

# 2. Ensure the row indices of the two tables are aligned
expr_raw = expr_raw.sort_index()
anno = anno.sort_index()

# 3. Replace row names with gene symbols from annoData
gene_symbols = anno["Symbol"].astype(str)
expr_raw.index = gene_symbols

print("=== Preview with gene symbols (first 5 rows) ===")
print(expr_raw.head())


# 4. Transpose so that each row = sample and each column = gene
expr_T = expr_raw.T.copy()
expr_T.index.name = "SampleID"
expr_T.reset_index(inplace=True)

# 5. Save as the table
expr_T.to_csv("IMvigor210_expr_full_gene_matrix.csv", index=False)

print("\n=== Final expression matrix preview (SampleID + first 5 genes) ===")
print(expr_T.iloc[:5, :6])  # SampleID + First 5 gene
print("Shape:", expr_T.shape)
print("Columns:", expr_T.columns[:10].tolist())


In [ ]:
import pandas as pd

# Load expression matrix
expr_T = pd.read_csv("IMvigor210_expr_full_gene_matrix.csv")

# Select the first sample
sample_row = expr_T.iloc[0:1]


long_preview = sample_row.melt(
    id_vars="SampleID",
    var_name="Gene",
    value_name="Expression"
).head(12)


def blue_style(df):
    return (
        df.style
        .set_table_styles([
            {"selector": "th", "props": [("background-color", "#1f4e79"),
                                         ("color", "white"),
                                         ("font-weight", "bold"),
                                         ("border", "1px solid #1f4e79")]},
            {"selector": "td", "props": [("border", "1px solid #1f4e79"),
                                         ("padding", "6px")]},
            {"selector": "table", "props": [("border-collapse", "collapse"),
                                            ("margin", "10px 0")]}
        ])
        .set_properties(**{
            "background-color": "#d9e2f3",
            "color": "black",
            "font-size": "13px"
        })
    )

blue_style(long_preview)


In [ ]:
import pandas as pd

# Load the pheno_fixed file generated in Step 1
pheno = pd.read_csv("IMvigor210_pheno_fixed.csv")

# Show the First 10 row
preview = pheno.head(10)

def blue_style(df):
    return (
        df.style
        .set_table_styles([
            {"selector": "th", "props": [("background-color", "#1f4e79"),
                                         ("color", "white"),
                                         ("font-weight", "bold"),
                                         ("border", "1px solid #1f4e79")]},
            {"selector": "td", "props": [("border", "1px solid #1f4e79"),
                                         ("padding", "4px")]},
            {"selector": "table", "props": [("border-collapse", "collapse")]}
        ])
        .set_properties(**{
            "background-color": "#d9e2f3",
            "color": "black",
            "font-size": "13px"
        })
    )

blue_style(preview)


In [ ]:
import pandas as pd
pheno = pd.read_csv("IMvigor210_pheno_fixed.csv")

row = pheno.iloc[0]

pheno_long = pd.DataFrame({
    "Feature": row.index,
    "Value": row.values
})

def blue_style(df):
    return (
        df.style
        .set_table_styles([
            {"selector": "th", "props": [
                ("background-color", "#1f4e79"),
                ("color", "white"),
                ("font-weight", "bold"),
                ("border", "1px solid #1f4e79"),
                ("padding", "6px")
            ]},
            {"selector": "td", "props": [
                ("border", "1px solid #1f4e79"),
                ("padding", "6px"),
                ("background-color", "#d9e2f3")
            ]}
        ])
        .set_properties(**{
            "color": "black",
            "font-size": "13px"
        })
    )

blue_style(pheno_long.head(20))


**step 2**

In [ ]:
import pandas as pd
import numpy as np

pheno_path = "/content/IMvigor210_pheno_fixed.csv"
pheno = pd.read_csv(pheno_path)

col_map = {
    "Age":        "Sample age",
    "ECOG":       "Baseline ECOG Score",
    "PDL1_IC":    "IC Level",
    "PDL1_TC":    "TC Level",
    "TMB":        "FMOne mutation burden per MB",
    "Smoking":    "Tobacco Use History",
    "time":       "os",
    "censOS":     "censOS"
}


keep_cols = ["SampleID"] + [v for v in col_map.values() if v in pheno.columns]
clin = pheno[keep_cols].copy()

rename_dict = {v: k for k, v in col_map.items() if v in clin.columns}
clin.rename(columns=rename_dict, inplace=True)

for c in ["Age", "ECOG", "TMB"]:
    if c in clin.columns:
        clin[c] = pd.to_numeric(clin[c], errors="coerce")

for c in ["Age", "ECOG", "TMB"]:
    if c in clin.columns:
        mean = clin[c].mean(skipna=True)
        std  = clin[c].std(skipna=True)
        clin[c + "_z"] = (clin[c] - mean) / std

#  Build survival labels
if "time" in clin.columns and "censOS" in clin.columns:
    clin["censOS"] = pd.to_numeric(clin["censOS"], errors="coerce").fillna(1).astype(int)
    clin["event"] = 1 - clin["censOS"]
else:
    raise ValueError("Miss 'os' or 'censOS' column，Please check phenoData。")

# clean categorical variables, and apply one-hot encoding
cat_cols = []
for c in ["PDL1_IC", "PDL1_TC", "Smoking"]:
    if c in clin.columns:
        cat_cols.append(c)
        clin[c] = clin[c].astype(str).str.strip().replace({"nan": np.nan})

clin_encoded = pd.get_dummies(clin, columns=cat_cols, drop_first=False, dummy_na=False)


out_path = "IMvigor210_clinical_step2_processed.csv"
clin_encoded.to_csv(out_path, index=False, encoding="utf-8")

print("Number of samples × Number of features：", clin_encoded.shape)

In [ ]:
import pandas as pd


clin = pd.read_csv("IMvigor210_clinical_step2_processed.csv")


preview = clin.head(5)

def blue_style(df):
    return (
        df.style
        .set_table_styles([
            {"selector": "th", "props": [
                ("background-color", "#1f4e79"),
                ("color", "white"),
                ("font-weight", "bold"),
                ("border", "1px solid #1f4e79"),
                ("padding", "6px")
            ]},
            {"selector": "td", "props": [
                ("border", "1px solid #1f4e79"),
                ("padding", "6px"),
                ("background-color", "#d9e2f3")
            ]},
        ])
        .set_properties(**{
            "color": "black",
            "font-size": "13px"
        })
    )

blue_style(preview)


In [ ]:
row = clin.iloc[0]
clin_long = pd.DataFrame({"Feature": row.index, "Value": row.values})

blue_style(clin_long.head(25))


**step 3**

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

expre_csv = "/content/IMvigor210_expr_fixed.csv"


TPM_MIN = 1.0
FRACTION_MIN = 0.10
TOP_K = 1000

expr = pd.read_csv(expre_csv, index_col=0)
expr = expr.apply(pd.to_numeric, errors='coerce')

# log1p transformation
expr_log1p = np.log1p(expr)

# Low-expression filtering
mask_keep = (expr > TPM_MIN).sum(axis=1) >= max(1, int(FRACTION_MIN * expr.shape[1]))
expr_filtered = expr_log1p.loc[mask_keep]

# Sort genes by variance
vars_ = expr_filtered.var(axis=1).sort_values(ascending=False)
keep_top = vars_.head(min(TOP_K, expr_filtered.shape[0])).index

expr_top1000 = expr_filtered.loc[keep_top]

out_path = "rnaseq_top1000_by_variance.csv"
expr_top1000.to_csv(out_path)
print("Final top-1000:", expr_top1000.shape)

In [ ]:
import pandas as pd

rna_top = pd.read_csv("rnaseq_top1000_by_variance.csv", index_col=0)

print("Top1000 RNA-seq matrix preview (first 10 genes × first 5 samples):")
print(rna_top.iloc[:10, :5])
print("Shape:", rna_top.shape)


In [ ]:
import pandas as pd


df = pd.read_csv("rnaseq_top1000_by_variance.csv", index_col=0)

df_show = df.head(20)

def blue_style(s):
    return ['background-color: #0A3D62; color: white; font-weight: bold;'
            'border: 1px solid white; text-align: center;'] * len(s)

styled = df_show.style.apply(blue_style, axis=1)

styled


**step 4**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

clinical_path = "/content/IMvigor210_clinical_step2_processed.csv"  # Step 2 Output
rnaseq_path   = "/content/rnaseq_top1000_by_variance.csv"  # Step 3 Output

clinical = pd.read_csv(clinical_path)

# Convert to string consistently
clinical["SampleID"] = clinical["SampleID"].astype(str).str.strip()

rnaseq = pd.read_csv(rnaseq_path)

rnaseq["SampleID"] = rnaseq["SampleID"].astype(str).str.strip()
rnaseq = rnaseq.set_index("SampleID")


rna_ids  = rnaseq.index.astype(str)
clin_ids = clinical["SampleID"].astype(str)

common_ids = sorted(set(rna_ids).intersection(set(clin_ids)))


# Reorder according to common_ids to ensure the two tables correspond one-to-one
clinical_sub = (
    clinical
    .set_index("SampleID")
    .loc[common_ids]
)
rnaseq_sub = rnaseq.loc[common_ids]


y_time  = clinical_sub["time"].astype(float).values
y_event = clinical_sub["event"].astype(int).values

# Build the final feature matrix X
drop_cols = [c for c in ["time", "event", "censOS"] if c in clinical_sub.columns]
clinical_features = clinical_sub.drop(columns=drop_cols)

X = pd.concat([clinical_features, rnaseq_sub], axis=1)

print(f"Number of samples after integration: {X.shape[0]}, number of features: {X.shape[1]}")

# Save the integrated features and labels (in the current directory)

features_out = "IMvigor210_features_step4.csv"
labels_out   = "IMvigor210_labels_step4.csv"

X.to_csv(features_out)
pd.DataFrame({
    "SampleID": common_ids,
    "time": y_time,
    "event": y_event
}).to_csv(labels_out, index=False)

print(f"Integrated features saved to: {features_out}")
print(f"Labels saved to: {labels_out}")

# 5-fold cross-validation split (patient-level)
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

fold_assign = pd.DataFrame({
    "SampleID": common_ids,
    "fold": -1
})

for fold_id, (_, val_idx) in enumerate(skf.split(X.values, y_event), start=1):
    val_ids = [common_ids[i] for i in val_idx]
    fold_assign.loc[fold_assign["SampleID"].isin(val_ids), "fold"] = fold_id


folds_out = "IMvigor210_cv5_folds_step4.csv"
fold_assign.to_csv(folds_out, index=False)

print(f"5-fold splits saved to: {folds_out}")



In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Read step4 Output
X = pd.read_csv("IMvigor210_features_step4.csv", index_col=0)
labels = pd.read_csv("IMvigor210_labels_step4.csv")
time_os = labels["time"].values
event = labels["event"].values

# Assume the first few columns are clinical variables and the remaining ones are RNA
clin_cols = [c for c in X.columns if c.startswith("Age") or c.startswith("ECOG") or "PDL1" in c or "Smoking" in c or "TMB" in c]
rna_cols  = [c for c in X.columns if c not in clin_cols]

X_clin = X[clin_cols]
X_rna  = X[rna_cols]

scaler = StandardScaler()
X_rna_scaled = scaler.fit_transform(X_rna)

pca = PCA(n_components=70)
X_rna_pca = pca.fit_transform(X_rna_scaled)

X_pca = pd.concat(
    [X_clin.reset_index(drop=True),
     pd.DataFrame(X_rna_pca, columns=[f"PC{i+1}" for i in range(X_rna_pca.shape[1])])],
    axis=1
)
X_pca.to_csv("IMvigor210_features_step4_PCA50.csv")

**step 5**

In [ ]:
import time
import pandas as pd
import numpy as np
import lightgbm as lgb

# Read Step4 Output
X = pd.read_csv("IMvigor210_features_step4_PCA50.csv", index_col=0)
labels = pd.read_csv("IMvigor210_labels_step4.csv")
folds = pd.read_csv("IMvigor210_cv5_folds_step4.csv")

ids = labels["SampleID"].tolist()
time_os = labels["time"].values
event = labels["event"].values

y = np.log(time_os + 1e-6)

print(f"Number of samples: {X.shape[0]}, number of features: {X.shape[1]}")

# Hyperparameters: medium grid (9 combinations)
num_leaves_list = [31, 63, 127]
learning_rate_list = [0.01, 0.03, 0.05]

medium_param_list = []

for nl in num_leaves_list:
    for lr in learning_rate_list:
        medium_param_list.append({
            "num_leaves": nl,
            "learning_rate": lr,
            "min_data_in_leaf": 20,
            "max_depth": -1,
            "lambda_l1": 0.0,
            "lambda_l2": 0.1,
            "subsample": 0.8,
            "colsample_bytree": 0.8,
            "n_estimators": 1000,
        })

print("Number of hyperparameter combinations:", len(medium_param_list))

# 5-fold CV training (timed)
oof_pred = np.zeros(len(X))
model_records = []

global_start = time.time()

for fold in range(1, 6):

    fold_start = time.time()

    val_mask = folds["fold"] == fold
    train_mask = ~val_mask

    train_idx = folds.index[train_mask]
    val_idx   = folds.index[val_mask]

    X_train = X.iloc[train_idx]
    X_val   = X.iloc[val_idx]

    y_train = y[train_idx]
    y_val   = y[val_idx]

    train_set = lgb.Dataset(X_train, label=y_train)
    val_set   = lgb.Dataset(X_val, label=y_val)

    best_rmse = np.inf
    best_model = None
    best_param = None

    # Hyperparameter search (9 combinations)
    for params in medium_param_list:
        p = params.copy()
        p.update({
            "objective": "regression",
            "metric": "rmse",
            "verbosity": -1,
            "num_threads": 4,
        })

        model = lgb.train(
            p,
            train_set,
            valid_sets=[val_set],
            num_boost_round=5000,
            callbacks=[
                lgb.early_stopping(stopping_rounds=200, verbose=False)
            ]
        )

        rmse = model.best_score["valid_0"]["rmse"]

        if rmse < best_rmse:
            best_rmse = rmse
            best_model = model
            best_param = p

    fold_time = time.time() - fold_start
    print(f"Fold {fold} best RMSE = {best_rmse:.4f}, elapsed time = {fold_time:.1f} s")

    model_records.append(best_param)
    oof_pred[val_idx] = best_model.predict(X_val, num_iteration=best_model.best_iteration)


# Derived Risk Score = -pred
risk_score = -oof_pred

out_df = pd.DataFrame({
    "SampleID": ids,
    "predicted_log_survival": oof_pred,
    "risk_score": risk_score,
})
out_df.to_csv("IMvigor210_step5_lightgbm_medium_regression_predictions.csv",
              index=False)

pd.DataFrame(model_records).to_csv(
    "IMvigor210_step5_medium_best_params_regression.csv",
    index=False
)

**step 6**

In [ ]:
# Step 6 – Evaluation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.utils import concordance_index

pred = pd.read_csv("IMvigor210_step5_lightgbm_medium_regression_predictions.csv")
labels = pd.read_csv("IMvigor210_labels_step4.csv")

df = pred.merge(labels, on="SampleID")

risk = df["risk_score"].values
time_os = df["time"].values
event = df["event"].values

print("Number of samples:", len(df))


cindex = concordance_index(time_os, -risk, event)


print(f"C-index = {cindex:.4f}")

with open("IMvigor210_step6_Cindex.txt", "w") as f:
    f.write(f"C-index = {cindex:.4f}")


# Split into high-risk and low-risk groups by median risk score
median_risk = np.median(risk)
group = (risk >= median_risk).astype(int)

df["risk_group"] = group

kmf = KaplanMeierFitter()

plt.figure(figsize=(7,5))

# Low risk group
kmf.fit(
    durations=df[group==0]["time"],
    event_observed=df[group==0]["event"],
    label="Low Risk"
)
kmf.plot(ci_show= False)

# High risk group
kmf.fit(
    durations=df[group==1]["time"],
    event_observed=df[group==1]["event"],
    label="High Risk"
)
kmf.plot(ci_show= False)

plt.title("Kaplan–Meier Curve by Predicted Risk Groups")
plt.xlabel("Time (days)")
plt.ylabel("Survival Probability")
plt.grid(alpha=0.3)

plt.savefig("IMvigor210_step6_KM_curve.png", dpi=300)
plt.show()

# ------- log-rank test -------
results = logrank_test(
    df[group==0]["time"],
    df[group==1]["time"],
    df[group==0]["event"],
    df[group==1]["event"]
)

print(f"p-value = {results.p_value:.4e}")

with open("IMvigor210_step6_logrank_p.txt", "w") as f:
    f.write(f"log-rank p-value = {results.p_value:.4e}")

**Survival Time Distribution & Event Counts**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

labels = pd.read_csv("IMvigor210_labels_step4.csv")

plt.figure(figsize=(7,5), dpi=150)
plt.hist(labels["time"], bins=30, edgecolor="black", alpha=0.8)
plt.xlabel("Overall survival time (months)", fontsize=12)
plt.ylabel("Number of patients", fontsize=12)
plt.title("Distribution of survival times", fontsize=14)
plt.tick_params(axis="both", labelsize=11)
plt.tight_layout()
plt.show()

plt.figure(figsize=(5,5), dpi=150)
labels["event"].value_counts().sort_index().plot(kind="bar", edgecolor="black")
plt.xticks([0,1], ["Censored (0)", "Death (1)"], rotation=0, fontsize=11)
plt.ylabel("Number of patients", fontsize=12)
plt.title("Event vs censored", fontsize=14)
plt.tight_layout()
plt.show()


**Risk Score Distribution**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pred = pd.read_csv("IMvigor210_step5_lightgbm_medium_regression_predictions.csv")
labels = pd.read_csv("IMvigor210_labels_step4.csv")
df = pred.merge(labels, on="SampleID")

plt.figure(figsize=(7,5), dpi=150)
plt.hist(df["risk_score"], bins=30, edgecolor="black", alpha=0.8)
plt.xlabel("Risk score", fontsize=12)
plt.ylabel("Number of patients", fontsize=12)
plt.title("Risk score distribution", fontsize=14)
plt.tick_params(axis="both", labelsize=11)
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,5), dpi=150)
df.boxplot(column="risk_score", by="event")
plt.xticks([1,2], ["Censored (0)", "Death (1)"], fontsize=11)
plt.xlabel("Event", fontsize=12)
plt.ylabel("Risk score", fontsize=12)
plt.title("Risk score distribution by event", fontsize=14)
plt.suptitle("")
plt.tight_layout()
plt.show()